# Part 1: Performing Interactive Data Analytics

In this part, we load our data into Snowflake and work with it using [pandas on Snowflake](https://docs.snowflake.com/developer-guide/snowpark/python/pandas-on-snowflake).

## Adding Python Packages 🎒

Snowflake Notebooks comes pre-installed with common Python libraries for data science 🧪 and machine learning 🧠! If we are looking to use other packages, click on the `Packages` dropdown on the top right to add additional packages to your notebook.

For the purpose of this demo, let's add the `modin` package to use [pandas on Snowflake](https://docs.snowflake.com/developer-guide/snowpark/python/pandas-on-snowflake) to work with our data.

In [ ]:
import streamlit as st
import modin.pandas as pd
import snowflake.snowpark.modin.plugin

## Connecting to Snowflake 

To work with data in Snowflake, we need to first get a session variable to connect to Snowflake. Since we're already logged in to Snowflake Notebook, we can get the session variable directly through the active notebook session. The session variable is the entrypoint that gives us access to using Snowflake's Python API, including Snowpark.

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Bringing data into Snowflake

First, let's create an external state and upload the CSV file.


In [ ]:
-- Create a external stage 
CREATE OR REPLACE STAGE FROSTBYTES
    URL = 's3://sfquickstarts/frostbyte_tastybytes/';

In [ ]:
menu_item = pd.read_csv("@frostbytes/analytics/menu_item_aggregate_v.csv")
menu_item.head()

## Profiling and summary statistics
We can look at the size and overall descriptive statistics of our dataframe `menu_item`.  

In [ ]:
menu_item.shape

In [ ]:
menu_item.describe()

In [ ]:
import altair as alt
numeric_cols = ['PRICE', 'BASE_PRICE', 'COST_OF_GOODS_USD', 'COUNT_ORDERS', 'TOTAL_QUANTITY_SOLD']
cols = st.columns(len(numeric_cols))
for idx, col in enumerate(numeric_cols):
    with cols[idx]:
        chart = alt.Chart(menu_item).mark_bar().encode(
            alt.X(f'{col}:Q', bin=True, title=col),
            alt.Y('count():Q', title='Count'),
            tooltip=['count()']
        ).properties(
            width=200,  
            height=300,
            title=f'Distribution of {col}'
        ).configure_title(
            fontSize=14 
        )
        st.altair_chart(chart, use_container_width=True)


## Data Cleaning and Transformation

Now let's clean up the data by performing some data transformation.


In [ ]:
menu_item["DATE"] = pd.to_datetime(menu_item["DATE"])
menu_item.head(5)

Let's compute the total revenue by multiplying the two columns together. 

In [ ]:
menu_item["TOTAL_REVENUE"] = menu_item["TOTAL_QUANTITY_SOLD"]*menu_item["PRICE"]
menu_item.head(10)

In [ ]:
weekly_total = menu_item.groupby("DAY_OF_WEEK")["TOTAL_REVENUE"].sum()
weekly_total

Next, we want to classify `DAY_TYPE` weekday/weekend using a mapping dictionary.


In [ ]:
# Convert the series to a dataframe and reset index for Altair
weekly_total_df = weekly_total.reset_index()
day_type = {0: 'Weekend',  1: 'Weekday',  2: 'Weekday',  3: 'Weekday',  4: 'Weekday',  5: 'Weekday',  6: 'Weekend'}
# Add day type classification
weekly_total_df['DAY_TYPE'] = weekly_total_df['DAY_OF_WEEK'].map(day_type)

In [ ]:
chart = alt.Chart(weekly_total_df).mark_bar().encode(
    x=alt.X('DAY_OF_WEEK:O', title='Day of Week'),
    y=alt.Y('TOTAL_REVENUE:Q', title='Total Revenue'),
    color=alt.Color('DAY_TYPE:N', 
                    scale=alt.Scale(domain=['Weekday', 'Weekend'],
                                  range=['#1f77b4', '#ff7f0e'])),
).properties(
    width=600,
    height=400,
    title='Total Revenue by Day of Week (Weekday vs Weekend)'
)

# Display the chart
st.altair_chart(chart, use_container_width=True)


We are interested in looking specifically at the sales of Buffalo Mac & Cheese across different food trucks. 

In [ ]:
buffalo_mac_cheese = menu_item[menu_item["MENU_ITEM_NAME"]=="Buffalo Mac & Cheese"]
buffalo_mac_cheese

Next, we perform a join to combine our `buffalo_mac_cheese` dataframe with another dataframe `order_item`.

In [ ]:
order_item = pd.read_csv("@frostbytes/analytics/order_item_cost_agg_v.csv")
order_item

By looking at `order_item`, we see that it has separate month year columns, but `buffalo_mac_cheese` (from the original `menu_item`) has one combined `DATE`, so let's extract the year and month column.

In [ ]:
# Extract year and month to conform with data in `order_item`
buffalo_mac_cheese['YEAR'] = buffalo_mac_cheese['DATE'].dt.year
buffalo_mac_cheese['MONTH'] = buffalo_mac_cheese['DATE'].dt.month

Now let's `groupby` the year month and menu type.

In [ ]:
# Group by YEAR and MONTH
grouped_bmc = buffalo_mac_cheese.groupby(['YEAR', 'MONTH','MENU_TYPE_ID'])["COUNT_ORDERS","TOTAL_QUANTITY_SOLD"].sum().reset_index()
grouped_bmc

Now that we have the same join key on the two tables, we can merge the columns together.

In [ ]:
# Now merge with order_item on YEAR and MONTH
merged_df = grouped_bmc.merge(order_item, on=['YEAR', 'MONTH'])
merged_df

In [ ]:
st.markdown("This is how the dataframe size changed from performing the merge operation:")
st.markdown(f"order_item size: {order_item.shape} + grouped_bmc size: {grouped_bmc.shape} -> merged_df size: {merged_df.shape}")

Now, save the output dataframe as a **Snowflake** table. 

In [ ]:
merged_df.to_snowflake("cleaned_table", index= None, if_exists="replace")

Now we can query this table directly using SQL to verify that the table has been created.

In [ ]:
SELECT * FROM cleaned_table LIMIT 5; 